<a href="https://colab.research.google.com/github/snehadarshini-angolkar/nlp-positive-reframing/blob/main/t5_large_re.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install transformers

In [2]:
# from transformers import pipeline

In [3]:
# classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [6]:
# !pip install tqdm
# !pip install rouge
# !pip install transformers

!pip install datsets transformers[sentencepiece]
!pip install sentencepiece


ERROR: Could not find a version that satisfies the requirement datsets (from versions: none)
ERROR: No matching distribution found for datsets
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk import word_tokenize
import torch.nn.functional as F

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
original_file_path = "/content/drive/MyDrive/Purdue/NLP/nlp_project/output.txt"
df_original = pd.read_csv(original_file_path, sep="|", header=None, names=["original", "rephrased"])
df_copy_1 = df_original.copy()

In [4]:
df_copy_1.head()

,original,rephrased
0,I need to get away from this city for a bit. I...,I'm thankful for everything this city gave me!...
1,Hate when people try to guilt trip you into so...,I don’t like it when people try to guilt -trip...
2,Why does it have to be so hard to network if y...,I'm glad networking with gaming comes very eas...
3,Children with autism use stereotypic behavior ...,Children with autism tend to demonstrate stere...
4,I have less than 2 hours to get home and make ...,I have less than 2 hours to get home and make ...


In [5]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=100):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        original_text = self.dataframe.iloc[idx]['original']
        rephrased_text = self.dataframe.iloc[idx]['rephrased']

        # Tokenize the input and target text
        input_ids = self.tokenizer.encode(original_text, return_tensors="pt")[0]  # Extract the tensor from the dictionary
        target_ids = self.tokenizer.encode(rephrased_text, return_tensors="pt")[0]

        # Pad or truncate to the specified max_length
        input_ids = F.pad(input_ids, (0, self.max_length - input_ids.size(0)))
        target_ids = F.pad(target_ids, (0, self.max_length - target_ids.size(0)))

        return {'input_ids': input_ids, 'target_ids': target_ids}

In [6]:
def train_epoch(model, train_loader, optimizer, device):
    model.train()
    total_loss = 0.0
    num_batches = len(train_loader)

    for batch in tqdm(train_loader, desc="Training", leave=False):
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)

        optimizer.zero_grad()
        loss = model(input_ids, labels=target_ids).loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

In [7]:
def evaluate(model, test_loader, device):
    model.eval()
    total_loss = 0.0
    num_batches = len(test_loader)
    generated_sentences = []

    with torch.no_grad(), tqdm(total=num_batches, desc="Evaluating", leave=False) as pbar:
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            target_ids = batch['target_ids'].to(device)

            loss = model(input_ids, labels=target_ids).loss
            total_loss += loss.item()

            # Generate rephrased sentences for testing
            generated_ids = model.generate(input_ids, max_length=100, num_beams=5, length_penalty=0.6)
            generated_sentence = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            generated_sentences.append(generated_sentence)

            pbar.update(1)

    average_loss = total_loss / num_batches
    return average_loss, generated_sentences

In [8]:
model_name = "t5-large"
    # model_path = "fine_tuned_model"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use th

In [9]:
train_df, test_df = train_test_split(df_copy_1, test_size=0.1, random_state=42)

train_data = train_df.iloc[0:4000]
val_data = train_df.iloc[4000:6011]

train_dataset = CustomDataset(train_data, tokenizer)
test_dataset = CustomDataset(test_df, tokenizer)
val_dataset = CustomDataset(val_data, tokenizer)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=None)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, collate_fn=None)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False, collate_fn=None)

In [10]:
# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define fine-tuning configuration
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 2  # Set the number of epochs to 1

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [12]:
for epoch in range(num_epochs):
    # Training
    train_loss = train_epoch(model, train_loader, optimizer, device)
    print(f"Epoch {epoch + 1}, Train Loss: {train_loss}")

    # Evaluation loss
    test_loss, generated_sentences = evaluate(model, val_loader, device)
    print(f"Epoch {epoch + 1}, Test Loss: {test_loss}")


# Save the fine-tuned model
model.save_pretrained("fine_tuned_model_ep2_large")

Epoch 1, Train Loss: None


Epoch 1, Test Loss: 0.4664307073291917


Epoch 2, Train Loss: None


Epoch 2, Test Loss: 0.4635544852696167


In [16]:
# !pip install sacrebleu
!pip install rouge

In [17]:
import sacrebleu

from rouge import Rouge

In [18]:
model.eval()
generated_sentences = []
target_sentences = []

In [19]:
# Use tqdm for the progress bar
with tqdm(total=len(test_loader), desc="Evaluation", unit="batch") as progress_bar:
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            target_ids = batch['target_ids'].to(device)

            # Generate rephrased sentences
            generated_ids = model.generate(input_ids, max_length=100, num_beams=5, length_penalty=0.6)
            generated_sentence = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            generated_sentences.append(generated_sentence)

            # Decode target sentences
            target_sentence = tokenizer.decode(target_ids[0], skip_special_tokens=True)
            target_sentences.append(target_sentence)

            # Update the progress bar
            progress_bar.update(1)

Evaluation: 100%|██████████| 334/334 [08:19<00:00,  1.50s/batch]


In [20]:
# Calculate BLEU scores using sacrebleu
bleu_scores = sacrebleu.corpus_bleu(generated_sentences, [target_sentences])

# Calculate ROUGE scores using sacrerouge
rouge = Rouge()
rouge_scores = rouge.get_scores(generated_sentences, target_sentences)

In [21]:
print("BLEU Score:", bleu_scores.score)
print("ROUGE-1:", rouge_scores[0]['rouge-1']['f'])
print("ROUGE-2:", rouge_scores[0]['rouge-2']['f'])
print("ROUGE-L:", rouge_scores[0]['rouge-l']['f'])

BLEU Score: 12.031721171090163
ROUGE-1: 0.06060605572084521
ROUGE-2: 0.0
ROUGE-L: 0.06060605572084521


In [3]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 3.9 MB/s eta 0:00:00


### Testing flan-t5-base


In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [2]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", device_map="auto")



tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> Wie old sind Sie?</s>


In [4]:
input_text = "I'm at hour 51 of having a houseguest and have just hidden in my room for a half hour AFTER taking a shower."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")


<pad> I'm so tired</s>


In [5]:
input_text = "Today is a very sad day, I got fired. My wife died."
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

In [6]:

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


<pad> I'm sad.</s>
